# Liste doublement chainée

## Principe 

La liste doublement chainée étend le concept de liste en ajoutant un lien vers l'élément précédent à chaque maillon de la chaine.

In [29]:
class Maillon:
    def __init__(self, v, s = None, p = None):
        self.donnee = v
        self.suivant = s 
        if s: s.precedent = self 
        self.precedent = p 
        if p: p.suivant = self
            
    def __str__(self): 
        return "{}".format(self.donnee)

La classe liste a au moins pour attributs les liens vers les maillons en tête et en queue de liste 

In [30]:
# caché dans les slides

def __liste_double_str__(L):
    if L.tete == None: return "⌀"
    m = L.tete
    s = "⌀ ← "
    while m != None:
        s += m.__str__()
        s += " ⇄ " if m.suivant else " → ⌀"
        m = m.suivant
    return s

In [31]:
class ListeDouble:
    def __init__(self):
        self.tete = None
        self.queue = None
    
    __str__ = __liste_double_str__

## Opérations

Les opérations essentielles sont 

* indiquer la taille, indiquer si la liste est vide

* insérer et supprimer en tête

* insérer et supprimer en queue

* traverser et itérer en avant ou en arrière

* insérer et supprimer en position quelconque

### Taille et vide 

Une liste est vide si elle n'a pas de maillon en tête. 

In [32]:
def est_vide(L):
    return L.tete == None
    
ListeDouble.empty = est_vide

Retourner la taille d'une liste requiert 
* soit de stocker et maintenir cet information comme attribut de `ListeDouble`
* soit de parcourir toute la liste pour compter les maillons, une opération $\Theta(n)$

### Insertion en tête

En général, insérer en tête requiert de

* créer un nouveau maillon `M`
* faire de `M` la nouvelle tête
* lier `M.suivant` avec l'ancienne tête `T`
* lier `T.precedent` avec le nouveau maillon `M`

Dans le cas d'une liste vide, il n'y a pas d'ancienne tête. Il faut également modifier la queue qui est le même maillon que la queue pour la liste d'un seul élément créée.  

In [39]:
def inserer_en_tete(L,valeur):
    if est_vide(L):
        L.tete = L.queue = Maillon(valeur)
    else:
        L.tete = Maillon(valeur, s = L.tete)
        
ListeDouble.appendleft = inserer_en_tete

In [44]:
L = ListeDouble(); print(L)
for i in range(4):
    L.appendleft(i**2)
    print(L)

⌀
⌀ ← 0 → ⌀
⌀ ← 1 ⇄ 0 → ⌀
⌀ ← 4 ⇄ 1 ⇄ 0 → ⌀
⌀ ← 9 ⇄ 4 ⇄ 1 ⇄ 0 → ⌀


### Supprimer en tête

On ne peut évidemment pas supprimer en tête d'une liste vide. 

Sinon, il faut  
* remplacer la tête par le maillon suivant `M`
* annuler le lien `M.precedent` qui pointait vers le maillon supprimé 
* selon le langage, détruire le maillon supprimé

et vérifier si la suppression ne vide pas la liste

In [45]:
def supprimer_en_tete(L):
    if est_vide(L): raise IndexError()
        
    L.tete = L.tete.suivant
    if L.tete:
        L.tete.precedent = None
    else: # liste vide
        L.queue = None

ListeDouble.popleft = supprimer_en_tete

In [46]:
print(L)
for i in range(5):
    L.popleft()
    print(L)

⌀ ← 9 ⇄ 4 ⇄ 1 ⇄ 0 → ⌀
⌀ ← 4 ⇄ 1 ⇄ 0 → ⌀
⌀ ← 1 ⇄ 0 → ⌀
⌀ ← 0 → ⌀
⌀


IndexError: 

### Opérations en queue

Les opérations en queue s'écrire exactement pareil, en remplaçant `tete` par `queue` ainsi que `suivant` par `precedent` et réciproquement

In [47]:
def inserer_en_queue(self,valeur):
    if est_vide(L): L.queue = L.tete = Maillon(valeur)
    else:           L.queue = Maillon(valeur, p = L.queue)
        
ListeDouble.append = inserer_en_queue

In [48]:
def supprimer_en_queue(L):
    if est_vide(L): raise IndexError()
        
    L.queue = L.queue.precedent
    if L.queue: L.queue.suivant = None
    else:       L.tete = None

ListeDouble.pop = supprimer_en_queue

In [55]:
L = ListeDouble(); print(L)
for i in range(3):
    L.append(i*2)
    print(L)

⌀
⌀ ← 0 → ⌀
⌀ ← 0 ⇄ 2 → ⌀
⌀ ← 0 ⇄ 2 ⇄ 4 → ⌀


In [56]:
for i in range(4):
    L.pop()
    print(L)

⌀ ← 0 ⇄ 2 → ⌀
⌀ ← 0 → ⌀
⌀


IndexError: 

### Traverser

La liste doublement chainée peut être traversée dans les deux sens

In [61]:
def traverser(L):
    m = L.tete
    while m != None:
        print(m, end = " "); m = m.suivant
            
def traverser_inverse(L):
    m = L.queue
    while m != None:
        print(m, end = " "); m = m.precedent 

In [74]:
L = ListeDouble(); 
for i in range(4): L.append(i*3)
print(L)
print("en avant:   ",end=""); traverser(L)
print("\nen arrière: ",end=""); traverser_inverse(L)

⌀ ← 0 ⇄ 3 ⇄ 6 ⇄ 9 → ⌀
en avant:   0 3 6 9 
en arrière: 9 6 3 0 

### Itérer

Indépendamment des détails de langage du python, on peut fournir un interface d'itération élémentaire via trois fonctions

In [75]:
def debut(L):   return L.tete
def fin(L):     return None
def suivant(m): return m.suivant

elles nous permettent de traverser la liste avec une boucle `while` 

In [76]:
print(L)
m = debut(L)
while m != fin(L):
    print(m, end=" ")
    m = suivant(m)

⌀ ← 0 ⇄ 3 ⇄ 6 ⇄ 9 → ⌀
0 3 6 9 

On peut aussi itérer en arrière via 

In [77]:
def r_debut(L): return L.queue
def r_fin(L):   return None
def r_suivant(m): return m.precedent

In [78]:
m = r_debut(L)
while m != r_fin(L):
    print(m, end=" ")
    m = r_suivant(m)

9 6 3 0 

### Opérations en position quelconque

On peut utiliser les itérateurs comme paramètres pour indiquer les positions d'insertion et de suppression. Par exemple, la fonction suivant insère devant l'itérateur `pos` fournit en paramètre. 

In [102]:
def inserer_devant(L,pos,val):
    if pos == L.tete: inserer_en_tete(L,val)
    else:
        m = Maillon(val,p = pos.precedent, s = pos )
        
ListeDouble.inserer = inserer_devant

Le code qui suit insère la valeur -1 devant tous les éléments pairs

In [107]:
L = ListeDouble(); 
for i in range(4): L.append(i*3)
print(L)

m = debut(L)
while m != fin(L):
    if m.donnee % 2 == 0:
        L.inserer(m,-1)
    m = suivant(m)
    
print(L)

⌀ ← 0 ⇄ 3 ⇄ 6 ⇄ 9 → ⌀
⌀ ← -1 ⇄ 0 ⇄ 3 ⇄ -1 ⇄ 6 ⇄ 9 → ⌀


Cependant, ce code a un défaut majeur, La fonction `inserer_devant` ne permet pas d'insérer en queue, même en écrivant `inserer_devant(L,fin(L),val)`. En effet, `fin(L)` retourne `None`

Une solution fréquemment utilisée est de d'utiliser un maillon sentinelle.

## Liste double 

In [ ]:
class ListeDouble2:
    def __init__(self):
        self.sentinelle = Maillon("rien")
    
    __str__ = __liste_double_str__

In [ ]:
def est_vide2(L):
    return L.tete == None
    
ListeDouble.empty = est_vide

<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>